In [1]:
import boa

In [19]:
tokens = ["USD", "EUR"]
erc20_list = [None] * len(tokens)
erc4626_list = [None] * len(tokens)
with boa.env.prank("0x0000000000000000000000000000000000001234"):
    for i in range(len(tokens)):
        erc20_list[i] = boa.load("contracts/testing/ERC20Mock.vy", token, token, 18)
        erc4626_list[i] = boa.load("contracts/testing/ERC4626Mock.vy", erc20_list[i].address)

In [20]:
mint_quantity = 10**6 * 10**18
for  i in range(len(tokens)):
    

0x0000000000000000000000000000000000000082
0x0000000000000000000000000000000000000084
0x0000000000000000000000000000000000000083
0x0000000000000000000000000000000000000085
